# Objetivo

El objetivo de este notebook de Colab es presentar una interfaz gráfica para poder ejecutar un modelo predictivo de forma sencilla, para el proyecto final de la certificación (hackaton)

>18/09/24
>
>Marco Mongi
>
>DS2402

In [17]:
# Para tener compatibilidad con la versión del modelo que guardé
!pip install scikit-learn==1.5.2
import sklearn
print(sklearn.__version__)

# 1: Predicciones a futuro
Le paso fecha de inicio y fecha de fin (cualquiera) y el modelo predice consumo en ese rango.

## Cargo modelo entrenado

In [18]:
# Code to read model file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link='https://drive.google.com/file/d/1I59yFJNMDwYV-8CAGoQStoNKR7KTcg7Y'
fluff, id = link.split('d/')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('model.joblib')

In [19]:
from joblib import load

# Cargar el modelo desde el archivo
model = load('model.joblib')

## Gradio

In [3]:
!pip install gradio > /dev/null 2>&1 # No quiero que se vea por consola

In [23]:
import gradio as gr
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

def predict_range(start_date, end_date):
    # Convertir las fechas de texto a objetos datetime
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    # Generar un rango de fechas con intervalos de 1 hora
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Crear el DataFrame
    df = pd.DataFrame(index=date_range)

    # Agregar las columnas hour, day_of_week y month
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month

    predictions_df = model.predict(df)

    plt.figure(figsize=(10, 5))
    plt.plot(date_range, predictions_df)

    # Rellenar el área bajo la curva con color azul y transparencia
    plt.fill_between(date_range, predictions_df, color='blue', alpha=0.3)

    #plt.title('Predicción de consumo')
    plt.xlabel('Fecha y Hora')
    plt.ylabel('Consumo (kW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ajusta el layout para que no se corte el gráfico

    return plt

# Definir la interfaz de Gradio
with gr.Blocks() as iface:
    gr.Markdown("# Predicción de consumo energético")
    gr.Markdown("Selecciona un rango de fechas para visualizar la predicción del consumo energético.")

    with gr.Row():
        start_date = gr.Textbox(label="Start Date", placeholder="YYYY-MM-DD (e.g. 2006-12-16)")
        end_date = gr.Textbox(label="End Date", placeholder="YYYY-MM-DD (e.g. 2010-11-26)")

    plot_output = gr.Plot()

    def update_plot(start_date, end_date):
        return predict_range(start_date, end_date)

    submit_button = gr.Button("Generar Gráfico")
    submit_button.click(update_plot, inputs=[start_date, end_date], outputs=plot_output)

# Ejecutar la aplicación
iface.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://586ef665778a94de32.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://586ef665778a94de32.gradio.live


# 2: Datos reales vs predicciones

In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt  # Importamos pyplot para graficar


## Cargo el dataset desde drive
Va a pedir acceso a la cuenta de google
Lo siguiente lee un archivo desde drive, y carga el archivo Dataset.csv en el entorno de ejecución

In [25]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link='https://drive.google.com/file/d/1NboG0z-qVZuTx0qKMScZFm0UPJt_Do4_'
fluff, id = link.split('d/')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('processed_dataset.csv')

In [26]:
# Asigno el contenido del csv a un dataframe
df = pd.read_csv('processed_dataset.csv',sep=',')

## Proceso dataset

In [27]:
# Cambiamos los tipos de datos para asegurar el manejo adecuado de las columnas
df = df.astype({
    'date': 'datetime64[ns]',  # Fecha y hora exacta de la medición
    'Consumption': float,      # Consumo energético en kW
    'hour': int,               # Hora del día de la medición
    'day_of_week': int,        # Día de la semana
    'month': int               # Mes del año
})

In [28]:
# Resampling
freq='1H'
df=df.resample(freq,on='date').mean()
df.reset_index(inplace=True)

In [29]:
df.set_index('date', inplace=True)

In [30]:
# Divido dataset en variables a predecir y predictores
X = df[['hour', 'day_of_week', 'month']] # Predictores
y = df['Consumption'] # Variable a predecir

## Prediccion de todo el dataset

In [31]:
# Predicciones
predictions = model.predict(X)

# Creamos un DataFrame para las predicciones con el índice temporal correcto
predictions_df = pd.DataFrame(data=predictions, index=X.index, columns=['Predicted'])

## Gradio
Como estoy comparando valores reales vs predicciones, el rango de fechas a usar debe ser el mismo que el del dataset.

Debo pasar fechas entre:
* 2006-12-16
* 2010-11-26

In [32]:
import gradio as gr
from datetime import datetime
import matplotlib.pyplot as plt  # Asegúrate de importar matplotlib

def plot_predictions(start_date, end_date):
    # Convertir las fechas de texto a objetos datetime
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    # Filtrar los datos según las fechas de entrada
    filtered_y_test = y[start_date:end_date]
    filtered_predictions = predictions_df.loc[start_date:end_date]

    # Crear el gráfico
    plt.figure(figsize=(10, 5))
    plt.plot(filtered_y_test, label='Real Consumption', color='blue')
    plt.plot(filtered_predictions, label='Predicted Consumption', color='red', linestyle='--')
    plt.title('Consumo Real vs. Predicción')
    plt.xlabel('Fecha y Hora')
    plt.ylabel('Consumo (kW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ajusta el layout para que no se corte el gráfico
    return plt

# Definir la interfaz de Gradio
with gr.Blocks() as iface:
    gr.Markdown("# Consumo Energético: Real vs Predicción")
    gr.Markdown("Selecciona un rango de fechas para visualizar el consumo energético real versus las predicciones.")

    with gr.Row():
        start_date = gr.Textbox(label="Start Date", placeholder="YYYY-MM-DD (e.g. 2006-12-16)")
        end_date = gr.Textbox(label="End Date", placeholder="YYYY-MM-DD (e.g. 2010-11-26)")

    plot_output = gr.Plot()

    def update_plot(start_date, end_date):
        return plot_predictions(start_date, end_date)

    submit_button = gr.Button("Generar Gráfico")
    submit_button.click(update_plot, inputs=[start_date, end_date], outputs=plot_output)

# Ejecutar la aplicación
iface.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1cbbd19dfd6f314cdf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1cbbd19dfd6f314cdf.gradio.live
